# Where's the spot?

An example of how to solve for the location, amplitude, and size of a star spot.

As we discuss in [this notebook](StarSpots.ipynb), `starry` isn't really meant for modeling discrete features such as star spots; rather, `starry` employs spherical harmonics to model the surface brightness distribution as a smooth, continuous function. We generally recommend approaching the mapping problem in this fashion; see the Eclipsing Binary tutorials for more information on how to do this. However, if you really want to model the surface of a star with star spots, read on!

Let's begin by importing stuff as usual:

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import numpy as np
import starry
import exoplanet as xo
import pymc3 as pm
import pymc3_ext as pmx
import matplotlib.pyplot as plt
from corner import corner

starry.config.quiet = True

We discussed how to add star spots to a `starry` map in [this tutorial](StarSpots.ipynb). Here, we'll generate a synthetic light curve from a star with a single large spot with the following parameters:

In [ ]:
# True values
truth = dict(contrast=0.25, radius=20, lat=30, lon=30)

Those are, respectively, the fractional amplitude of the spot, its standard deviation (recall that the spot is modeled as a Gaussian in $\cos\Delta\theta$, its latitude and its longitude.

To make things simple, we'll assume we know the inclination and period of the star exactly:

In [ ]:
# Things we'll assume are known
inc = 60.0
P = 1.0

Let's instantiate a 15th degree map and give it those properties:

In [ ]:
map = starry.Map(15)
map.inc = inc
map.spot(
    contrast=truth["contrast"],
    radius=truth["radius"],
    lat=truth["lat"],
    lon=truth["lon"],
)
map.show()

Now we'll generate a synthetic light curve with some noise:

In [ ]:
t = np.linspace(0, 3.0, 500)
flux0 = map.flux(theta=360.0 / P * t).eval()
np.random.seed(0)
flux_err = 2e-4
flux = flux0 + flux_err * np.random.randn(len(t))

Here's what that looks like:

In [ ]:
plt.plot(t, flux)
plt.xlabel("time [days]")
plt.ylabel("normalized flux");

In this notebook, we are going to derive posterior constraints on the spot properties. Let's define a `pymc3` model and within it, our priors and flux model:

In [ ]:
with pm.Model() as model:

    # Priors
    contrast = pm.Uniform("contrast", lower=0.0, upper=1.0, testval=0.5)
    radius = pm.Uniform("radius", lower=10.0, upper=35.0, testval=15.0)
    lat = pm.Uniform("lat", lower=-90.0, upper=90.0, testval=0.1)
    lon = pm.Uniform("lon", lower=-180.0, upper=180.0, testval=0.1)

    # Instantiate the map and add the spot
    map = starry.Map(ydeg=15)
    map.inc = inc
    map.spot(contrast=contrast, radius=radius, lat=lat, lon=lon)

    # Compute the flux model
    flux_model = map.flux(theta=360.0 / P * t)
    pm.Deterministic("flux_model", flux_model)

    # Save our initial guess
    flux_model_guess = pmx.eval_in_model(flux_model)

    # The likelihood function assuming known Gaussian uncertainty
    pm.Normal("obs", mu=flux_model, sd=flux_err, observed=flux)

We've placed some generous uniform priors on the four quantities we're solving for. Let's run a quick gradient descent to get a good starting position for the sampler:

In [ ]:
with model:
    map_soln = pmx.optimize()(start=model.test_point)

Here's the data and our initial guess before and after the optimization:

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(t, flux, "k.", alpha=0.3, ms=2, label="data")
plt.plot(t, flux_model_guess, "C1--", lw=1, alpha=0.5, label="Initial")
plt.plot(
    t, pmx.eval_in_model(flux_model, map_soln, model=model), "C1-", label="MAP", lw=1
)
plt.legend(fontsize=10, numpoints=5)
plt.xlabel("time [days]", fontsize=24)
plt.ylabel("relative flux", fontsize=24);

And here are the maximum a posteriori (MAP) values next to the true values:

In [ ]:
print("{0:12s} {1:10s} {2:10s}".format("", "truth", "map_soln"))
for key in truth.keys():
    print("{0:10s} {1:10.5f} {2:10.5f}".format(key, truth[key], map_soln[key]))

Not bad! Looks like we recovered the correct spot properties. But we're not done! Let's get posterior constraints on them by sampling with `pymc3`. Since this is such a simple problem, the following cell should run in about a minute:

In [ ]:
with model:
    trace = pmx.sample(tune=250, draws=500, start=map_soln, chains=4, target_accept=0.9)

Plot some diagnostics to assess convergence:

In [ ]:
var_names = ["contrast", "radius", "lat", "lon"]
display(pm.summary(trace, var_names=var_names))

And finally, the corner plot showing the joint posteriors and the true values (in blue):

In [ ]:
samples = pm.trace_to_dataframe(trace, varnames=var_names)
corner(samples, truths=[truth[name] for name in var_names]);

We're done! It's easy to extend this to multiple spots, simply by calling `map.spot` once for each spot in the model, making sure you define new `pymc3` variables for the spot properties of each spot.